In [16]:
from cube import Cube
from tile import Tile
from ck2map import CK2Map
import continent_gen
import random

In [29]:
def make_capital_county(c_size=5, coastal=True, shuffle=True):
    '''Makes a county where all provinces neighbor the central province.'''
    a = Tile()
    cube_list = list(Cube(0,0,0).neighbors())
    if shuffle:
        random.shuffle(cube_list)
    if coastal and c_size<7:
        a.water_list.append(cube_list[c_size-1])
    cube_list = cube_list[:c_size-1]
    for el in cube_list:
        a.add_hex(el)
    return a
        

def make_capital_duchy(origin=Cube(0,0,0), size_list=[5,4,4,4,4], coastal=True, shuffle=True):
    '''Makes a duchy where all provinces neighbor the central province.
    If coastal=True and d_size<7, one water hex bordering the capital will be included.
    To make a fidget spinner, use d_size=4 and shuffle=False.'''
    duchy = Tile(origin=origin, tile_list=[make_capital_county(c_size=size_list[0], coastal=coastal, shuffle=shuffle)],
                 hex_list=[])
    for el in size_list[1:]:
        duchy.add_new_tile(el)
    return duchy


def make_kingdom(origin=Cube(0,0,0), size_list = [[5,4,4,4,4], [4,4,3,3], [4,4,3,3], [4,4,3]]):
    kingdom = Tile(origin=origin, tile_list=[make_capital_duchy(size_list=size_list[0])], hex_list=[])
    for duchy_size_list in size_list[1:]:
        new_county = Tile.new_tile(county_size_list[0])
        new_duchy = Tile(origin=Cube(0,0,0), tile_list = [new_county], hex_list=[])
        if new_county.move_into_place(kingdom.neighbors(),
                                      kingdom.relative_hex_list(),
                                      kingdom.relative_water_list(),
                                     ):
            for county_size_list in duchy_size_list:
                
                
                kingdom.add_new_tile(el)
    return kingdom

In [26]:
coastal = True
shuffle = True
duchy = Tile(origin=origin, tile_list=[make_capital_county(c_size=size_list[0][0], coastal=coastal, shuffle=shuffle)],
                 hex_list=[])

In [30]:
kingdom = make_kingdom()
kingdom.doodle()

TypeError: '<' not supported between instances of 'int' and 'list'

In [22]:

origin=Cube(0,0,0)
size_list = [[5,4,4,4,4], [4,4,3,3], [4,4,3,3], [4,4,3]]
kingdom = Tile(origin=origin, tile_list=[make_capital_duchy(size_list=size_list[0])], hex_list=[])
kingdom.doodle()

AttributeError: 'NoneType' object has no attribute 'real_hex_list'

In [1]:
from math import sqrt
from PIL import Image, ImageDraw

In [ ]:
class TitleDoodler:
    def __init__(self, size=(500,500), radius=15):
        self.im = Image.new('RGB', size, (0, 0, 0))
        self.draw = ImageDraw.Draw(self.im)
        self.size = size
        self.radius = radius

    def get_center(self, cube):
        assert cube.x + cube.y + cube.z == 0, f'bad cube {cube}'
        p = np.array(self.size) / 2
        p += cube.x * np.array([sqrt(3), -1]) * self.radius
        p += cube.y * np.array([0, -1]) * self.radius * 2
        return p

    def get_hex_points(self, p, radius=None):
        radius = radius or self.radius
        points = []
        for angle in range(0, 360, 60):
            theta = angle * np.pi / 180
            delta = np.array([np.cos(theta), np.sin(theta)]) * radius
            points.append(tuple(p + delta))
        return points

    def draw_barony(self, cube, color):
        '''Return the '''
        self.draw.polygon(self.get_hex_points(self.get_center(cube)), fill=color)
        